In [ ]:
import xml.etree.ElementTree as ET 
import gzip
from xml.etree.cElementTree import ElementTree

In [ ]:
initialPlanFile = '/Users/MathildeMoliner/Documents/Polytechnique/MATSim/create_input/decompress/input_plans.xml.gz'
treePopulation = ET.parse(gzip.open(initialPlanFile,'r'))
population  = treePopulation.getroot()

In [ ]:
motorHousehold=[]
linkHousehold=[]

# Regarder si le ménage est motorisé  pour relever son householdId et linkHousehold
for person in population.iter('person'):
    carAvail=person[0][2]
    householdId=person[0][5]
    linkHome=person[1][0].get('link')
    if carAvail.text!='never':
        if householdId.text not in motorHousehold: 
            motorHousehold.append(householdId.text)
            linkHousehold.append(linkHome)

print('Nombre de PAV :', len(motorHousehold))

In [ ]:
#Modifier les plans pour retirer imposer les modes private_AV aux utilisateurs car et car_passenger
for person in population.iter('person'):
    carAvail=person[0][2]
    householdId=person[0][5]
    if householdId.text in motorHousehold: #si motorisé 
        for leg in person[1].iter('leg'):
            if leg.get('mode') == 'car':
                leg.set('mode','private_AV')
                leg[0][0].text='private_AV'
                if leg[1].get('start_link') == leg[1].get('end_link'): #éviter les erreurs car une requête start_link=end_link échoue
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
            if leg.get('mode') == 'car_passenger':
                leg.set('mode','private_AV')
                leg[0][0].text='private_AV'
                if leg[1].get('start_link') == leg[1].get('end_link'):
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
        subpop=ET.SubElement(person[0],'attribute', attrib={'name': "subpopulation", 'class':"java.lang.String"})
        subpop.text="private"
        carAvail.text='never' 
    else:        
        subpop=ET.SubElement(person[0],'attribute', attrib={'name': "subpopulation", 'class':"java.lang.String"})
        subpop.text="no_private"
        for leg in person[1].iter('leg'):
            if leg.get('mode') == 'car':
                leg.set('mode','walk')
                leg[0][0].text='walk'
                leg[1].text=''
            if leg.get('mode') == 'car_passenger':
                leg.set('mode','walk')
                leg[0][0].text='walk'
                leg[1].text=''

In [ ]:

modifiedPlanFile = 'data/scenario1/population_sce1.xml.gz'
f = gzip.open(modifiedPlanFile, 'wb')
f.write('<?xml version="1.0" encoding="utf-8"?><!DOCTYPE population SYSTEM "http://www.matsim.org/files/dtd/population_v6.dtd">'.encode('utf8'))
ElementTree(population).write(f) 
f.close()

In [ ]:
# Utiliser CreateTaxiVehicles.java pour créer le fichier de véhicules privés selon len(motorHousehold)
#  en modifiant: int numberofVehicles = 59502;
#                String networkfile = "/Users/MathildeMoliner/Documents/Polytechnique/MATSim/create_input/data/input_files/montreal_network.xml.gz";
#		         String taxisFile = "/Users/MathildeMoliner/Documents/Polytechnique/MATSim/create_input/decompress/private_vehicles_sce1.xml";
		
treeVehicles = ET.parse('./decompress/private_vehicles_sce1.xml')
rootVehicles = treeVehicles.getroot()

In [ ]:
i=0
for taxi in rootVehicles.iter('vehicle'):
    taxi.set('id','taxi_' + str(motorHousehold[i]))
    taxi.set('start_link',linkHousehold[i])
    i+=1

In [ ]:
g = gzip.open('data/scenario1/private_vehicles_sce1.xml.gz', 'wb')
g.write('<?xml version="1.0" encoding="utf-8"?><!DOCTYPE vehicles SYSTEM "http://matsim.org/files/dtd/dvrp_vehicles_v1.dtd">'.encode('utf8'))
ElementTree(rootVehicles).write(g,'utf-8')
g.close()